In [ ]:
# y_seq_len + merged_future_prediction
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import random
import os
import yfinance as yf
import timeit
import datetime
import calendar
import warnings
from torch.nn.modules.transformer import TransformerEncoderLayer, TransformerEncoder
from torch.nn.modules.transformer import TransformerDecoder, TransformerDecoderLayer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.cuda.amp import GradScaler, autocast
from sklearn.ensemble import BaggingClassifier


warnings.filterwarnings("ignore", category=FutureWarning)

def create_sequences(input_data, sequence_length):
    sequences = []
    for i in range(len(input_data) - sequence_length):
        seq = input_data[i:i+sequence_length]
        sequences.append(seq)
    return torch.stack(sequences)

# class DecoderOnlyTransformerModel(nn.Module):
#     def __init__(self, n_features, nhead=None, nhid=None, nlayers=None, dropout=None,
#                  l1_regularization=0, l2_regularization=0,
#                  activation_function=None):
#         super(DecoderOnlyTransformerModel, self).__init__()

#         self.pos_encoder = nn.Sequential(
#             nn.Linear(n_features, nhid),
#             activation_function,
#             nn.Linear(nhid, nhid),
#             activation_function
#         )

#         decoder_layers = TransformerDecoderLayer(nhid, nhead)
#         self.transformer_decoder = TransformerDecoder(decoder_layers,nlayers)

#         self.decoder = nn.Linear(nhid,n_features)

#         self.l1_regularization = l1_regularization
#         self.l2_regularization = l2_regularization

#     def init_weights(self):
#         # initrange = 0.1
#         nn.init.xavier_uniform_(self.pos_encoder[0].weight)
#         nn.init.xavier_uniform_(self.pos_encoder[2].weight)
#         self.decoder.bias.data.zero_()
#         nn.init.xavier_uniform_(self.decoder.weight)

#     def regularization_loss(self):
#         l1_loss = 0
#         l2_loss = 0
#         for param in self.parameters():
#             l1_loss += torch.norm(param, 1)
#             l2_loss += torch.norm(param, 2) ** 2
#         return self.l1_regularization * l1_loss + self.l2_regularization * l2_loss

#     def generate_square_subsequent_mask(self, sz):
#         mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
#         mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
#         return mask

#     def forward(self, x):
#         x = self.pos_encoder(x)

#         # Generating target mask
#         tgt_mask = self.generate_square_subsequent_mask(x.size(0)).to(x.device)

#         # Processing the entire input at once
#         output = self.transformer_decoder(x, x, tgt_mask=tgt_mask)

#         # Apply the final linear layer
#         output = self.decoder(output)
#         # Selecting the last element from each sequence
#         output = output[:, -1, :].unsqueeze(1)
#         return output
    
# class TransformerModel(nn.Module):

#     def __init__(self, n_features, nhead=None, nhid=None, nlayers=None, dropout=None, l1_regularization=0, l2_regularization=0, activation_function = None):
#         super(TransformerModel, self).__init__()

#         self.activation_function = activation_function

#         # Model architecture
#         self.pos_encoder = nn.Sequential(
#             nn.Linear(n_features, nhid),
#             self.activation_function,
#             nn.LayerNorm(nhid),
#             nn.Dropout(dropout),
#             nn.Linear(nhid, nhid),
#             self.activation_function,
#             nn.LayerNorm(nhid)
#         )
#         encoder_layers = TransformerEncoderLayer(nhid, nhead, nhid, dropout,)
#         # encoder_layers.self_attn.batch_first = True
#         self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
#         self.decoder = nn.Linear(nhid, n_features)
#         self.l1_regularization = l1_regularization
#         self.l2_regularization = l2_regularization
#         self.init_weights()

#     def init_weights(self):
#         for layer in self.pos_encoder:
#             if isinstance(layer, nn.Linear):
#                 nn.init.xavier_uniform_(layer.weight)

#         self.decoder.bias.data.zero_()
#         nn.init.xavier_uniform_(self.decoder.weight)

#     def regularization_loss(self):
#         l1_loss = 0
#         l2_loss = 0
#         for param in self.parameters():
#             l1_loss += torch.norm(param, 1)
#             l2_loss += torch.norm(param, 2) ** 2

#         return self.l1_regularization * l1_loss + self.l2_regularization * l2_loss

#     def forward(self, src):
#         src = self.pos_encoder(src)
#         output = self.transformer_encoder(src)
#         output = self.decoder(output)
#         output = output[:, -1:, :]
#         return output    

class CNNModel(nn.Module):
    def __init__(self, n_features, n_filters=64, filter_size=3, dropout=0.5, activation_function=None, l1_regularization=0, l2_regularization=0, sequence_length=5):
        super(CNNModel, self).__init__()

        self.l1_regularization = l1_regularization
        self.l2_regularization = l2_regularization
        self.activation = activation_function
        
        self.conv1 = nn.Conv1d(in_channels=n_features, out_channels=n_filters, kernel_size=min(filter_size, sequence_length), padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(n_filters, n_filters * 2, kernel_size=min(filter_size, sequence_length // 2), padding=1)  # Adjusted kernel size
        self.dropout = nn.Dropout(dropout)
        
        # Dynamically calculate the flattened size after convolutions and pooling
        test_input = torch.rand(1, n_features, sequence_length)  # Simulate a random input to the model
        test_output = self.pool(self.conv1(test_input))
        test_output = self.pool(self.conv2(test_output))
        self.flattened_size = test_output.numel() // test_output.shape[0]
        
        self.fc1 = nn.Linear(self.flattened_size, 100)
        self.fc2 = nn.Linear(100, n_features)

    def regularization_loss(self):
        l1_loss, l2_loss = 0, 0
        for param in self.parameters():
            l1_loss += torch.norm(param, 1)
            l2_loss += torch.norm(param, 2) ** 2
        return self.l1_regularization * l1_loss + self.l2_regularization * l2_loss

    def forward(self, x):
        x = x.transpose(1, 2)  # Adjusting input shape for Conv1d
        x = self.activation(self.conv1(x))
        x = self.pool(x)
        x = self.activation(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)  # Flatten the output for the dense layer
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = x.unsqueeze(1)
        return x

def train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=torch.optim.Adam, criterion=torch.nn.MSELoss, 
                batch_size=32, patience=10, min_delta=0.0001, learning_rate=1e-3, max_norm=1.0, nan_patience=1, num_workers=4, 
                pin_memory=False, validation_frequency=1, save_directory=None, trial=1, model_ver=None):
    # Setup the device (GPU or CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)  # Move the model to the specified device

    # Initialize the optimizer and loss function
    optimizer = optimizer(model.parameters(), lr=learning_rate)
    criterion = criterion()

    # Prepare DataLoader for training data
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                              num_workers=num_workers, pin_memory=pin_memory)

    # Prepare DataLoader for validation data
    valid_dataset = TensorDataset(X_valid, y_valid)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, 
                              num_workers=num_workers, pin_memory=pin_memory)

    # Initialize variables for early stopping and best validation loss tracking
    best_val_loss = float('inf')
    early_stopping_counter = 0
    nan_counter = 0  # Counter for epochs with NaN loss
    stopped_early = False  # Flag to indicate if training stopped early

    # Initialize gradient scaler for mixed precision training
    scaler = GradScaler()

    # Variables to store training and validation losses
    train_losses = []
    val_losses = []

    # Training loop
    for epoch in range(n_epochs):
        model.train()  # Set model to training mode
        epoch_train_losses = []  # List to store losses for each batch

        # Loop over batches of data
        for batch_X_train, batch_y_train in train_loader:
            batch_X_train, batch_y_train = batch_X_train.to(device), batch_y_train.to(device)

            optimizer.zero_grad()  # Clear previous gradients

            # Forward pass with autocast for mixed precision
            with autocast():
                output = model(batch_X_train)
                # Recalculate the loss with the correctly adjusted target tensor
                loss = criterion(output, batch_y_train)
                # Assert to ensure shapes match
                assert output.shape == batch_y_train.shape, "Shape mismatch between model output and targets"
                reg_loss = model.regularization_loss() if hasattr(model, 'regularization_loss') else 0
                total_loss = loss + reg_loss  # Combine loss with regularization loss

            if torch.isnan(loss):
                nan_counter += 1
                if nan_counter >= nan_patience:
                    print(f"Training stopped early at epoch {epoch} due to NaNs in loss")
                    stopped_early = True
                    break
            else:
                nan_counter = 0

            # Backward pass and optimize
            scaler.scale(total_loss).backward()  # Scale loss for mixed precision
            scaler.unscale_(optimizer)  # Unscale gradients for gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)  # Clip gradients
            scaler.step(optimizer)  # Optimizer step
            scaler.update()  # Update the scaler

            epoch_train_losses.append(total_loss.item())  # Store loss
            
        if stopped_early:
            break

        # Store average training loss for the epoch
        train_losses.append(np.mean(epoch_train_losses))

        # Validation phase
        if epoch % validation_frequency == 0:
            model.eval()  # Set model to evaluation mode
            epoch_val_losses = []  # List to store validation losses

            # Loop over batches of validation data
            with torch.no_grad():
                for batch_X_valid, batch_y_valid in valid_loader:
                    batch_X_valid, batch_y_valid = batch_X_valid.to(device), batch_y_valid.to(device)
                    valid_output = model(batch_X_valid)
                    val_loss = criterion(valid_output, batch_y_valid)
                    epoch_val_losses.append(val_loss.item())  # Store validation loss

            # Calculate average validation loss and update early stopping
            val_losses.append(np.mean(epoch_val_losses))
            print(f"Epoch {epoch}: Train Loss = {train_losses[-1]:.5f}, Val Loss = {val_losses[-1]:.5f}")

            # Check for validation loss improvement
            if val_losses[-1] < best_val_loss - min_delta:
                best_val_loss = val_losses[-1]  # Update best validation loss
                early_stopping_counter = 0  # Reset early stopping counter
                # Save model checkpoint if directory is provided
                # Save the model
                if save_directory:
                    save_path = os.path.join(save_directory, f"model_ver{model_ver}_trial{trial}_epoch{epoch}.pt")
                else:
                    save_path = f"model_trial_{trial}.pt"

                torch.save(model, save_path)
                print(f"Validation loss improved to {best_val_loss:.5f} at epoch {epoch}, saving model...")
            else:
                early_stopping_counter += 1  # Increment early stopping counter

            # Trigger early stopping if no improvement for 'patience' epochs
            if early_stopping_counter >= patience:
                print("Early stopping triggered due to no improvement in validation loss.")
                break  # Exit training loop

    return train_losses, val_losses, stopped_early  # Return loss history and early stopping flag

def plot_results(train_losses, val_losses, trial, save_directory=None):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses)
    plt.plot(val_losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train Loss', 'Valid Loss'])
    plt.title(f'Train and Valid Losses (Trial {trial+1})')

    if save_directory:
        save_path = os.path.join(save_directory, f"loss_plot_trial_{trial}.png")
        plt.savefig(save_path)

    plt.show()

# def model_save(n_trials=1, model_save=True, save_directory=None, plot_loss=True, output_file_path=None, model_ver=None):

#     if save_directory and not os.path.exists(save_directory):
#         os.makedirs(save_directory)

#     all_results_params = []

#     for trial in range(n_trials):
#         print(f"Trial {trial + 1} of {n_trials}")

#         start = timeit.default_timer()

#         # seq_len = random.choice(range(3, 6))
#         # # nhead = random.choice(range(20, 21))
#         # nhid = random.choice(range(5000, 5001))
#         # nlayers = random.choice(range(1, 2))
#         # # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
#         # # activation_function = random.choice([torch.nn.GELU(), torch.nn.SiLU()])
#         # # activation_function = random.choice([torch.nn.Tanh()])
#         # # activation_function = random.choice([torch.nn.Mish()])
#         # dropout = 0 * random.choice(range(10, 11))
#         # optimizer = random.choice([torch.optim.AdamW])
#         # criterion = random.choice([torch.nn.SmoothL1Loss])
#         # # criterion = random.choice([torch.nn.MSELoss, torch.nn.L1Loss, torch.nn.KLDivLoss])
#         # n_epochs = random.choice(range(300, 501))
#         # batch_size = random.choice(range(256, 257))
#         # learning_rate = 0.00001 * random.choice(range(10, 11))
#         # patience = random.choice(range(30, 31))
#         # min_delta = 0.00001 * random.choice(range(1, 2))
#         # l1_regularization = 0.0000001 * random.choice(range(4, 5))
#         # l2_regularization = l1_regularization

#         seq_len = random.choice(range(5, 6))
#         nhead = random.choice(range(21, 31))
#         nhid = nhead * random.choice(range(30, 31))
#         nlayers = random.choice(range(1, 2))
#         # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
#         # activation_function = random.choice([torch.nn.GELU(), torch.nn.SiLU()])
#         activation_function = random.choice([torch.nn.Tanh(), torch.nn.ELU(), torch.nn.Mish()])
#         # activation_function = random.choice([torch.nn.Mish()])
#         dropout = 0.01 * random.choice(range(5, 21))
#         optimizer = random.choice([torch.optim.Adam])
#         criterion = random.choice([torch.nn.SmoothL1Loss])
#         # criterion = random.choice([torch.nn.MSELoss, torch.nn.L1Loss, torch.nn.KLDivLoss, torch.nn.SmoothL1Loss])
#         n_epochs = random.choice(range(200, 301))
#         batch_size = random.choice(range(256, 257))
#         learning_rate = 0.00001 * random.choice(range(10, 11))
#         patience = random.choice(range(10, 11))
#         min_delta = 0.00001 * random.choice(range(1, 2))
#         l1_regularization = 0.0000001 * random.choice(range(1, 2))
#         l2_regularization = l1_regularization

#         # Load tensors
#         X_train = torch.load(os.path.join(output_file_path, 'X_train.pt'))
#         y_train = torch.load(os.path.join(output_file_path, 'y_train.pt'))
#         X_valid = torch.load(os.path.join(output_file_path, 'X_valid.pt'))
#         y_valid = torch.load(os.path.join(output_file_path, 'y_valid.pt'))

#         # Initialize the model
#         if model_ver == 'Transformer':
#             model = TransformerModel(n_features=X_train.shape[2], nhead=nhead, nhid=nhid, nlayers=nlayers, activation_function=activation_function, dropout=dropout,
#                 l1_regularization=l1_regularization, l2_regularization=l2_regularization)
#         elif model_ver == 'Decoder':
#             model = DecoderOnlyTransformerModel(n_features=X_train.shape[2], nhead=nhead, nhid=nhid, nlayers=nlayers, activation_function=activation_function, dropout=dropout,
#             l1_regularization=l1_regularization, l2_regularization=l2_regularization)

#         # Train the model
#         train_losses, val_losses, stopped_early = train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=optimizer, criterion=criterion,
#                                                               batch_size=batch_size, patience=patience, min_delta=min_delta, learning_rate=learning_rate, trial=trial, 
#                                                               model_ver=model_ver, save_directory=save_directory)
#         # Check if training stopped early due to NaNs or not
#         if stopped_early:
#             print(f"Random search iteration {trial+1} stopped early due to NaNs in loss")
#             # Using 'continue' here will skip the remaining statements of the current iteration and proceed to the next iteration
#             continue

#         if plot_loss:
#             plot_results(train_losses, val_losses, trial, save_directory=save_directory)

#         # # Save the model
#         # if model_save:
#         #     if save_directory:
#         #         save_path = os.path.join(save_directory, f"model_trial_{trial}.pt")
#         #     else:
#         #         save_path = f"model_trial_{trial}.pt"
#         #     torch.save(model, save_path)

#         # Add the results to the results dataframe
#         params = {"seq_len": seq_len, "nhead": nhead, "nhid": nhid, "nlayers": nlayers, "activation_function": activation_function,
#                 "optimizer": optimizer, "criterion": criterion, "dropout": dropout, "n_epochs": n_epochs,
#                 "batch_size": batch_size, "learning_rate": learning_rate, "patience": patience, "min_delta": min_delta,
#                 "l1_regularization": l1_regularization, "l2_regularization": l2_regularization,
#                 }

#         results_params = {**params, "trial": trial+1, "train loss": train_losses[-1], "valid loss": val_losses[-1]}

#         all_results_params.append(results_params)
#         all_results_params_df = pd.DataFrame(all_results_params)

#         if save_directory:
#             all_results_params_df.to_csv(os.path.join(save_directory, f"all_results_params_{trial}.csv"))

#         end = timeit.default_timer()
#         # Calculate and print duration
#         duration = end - start
#         print(f"Execution Time of Trial {trial + 1} of {n_trials} is: {duration} seconds")

#     return all_results_params_df

def model_save(n_trials=1, model_save=True, save_directory=None, plot_loss=True, output_file_path=None):

    if save_directory and not os.path.exists(save_directory):
        os.makedirs(save_directory)

    all_results_params = []

    for trial in range(n_trials):
        print(f"Trial {trial + 1} of {n_trials}")

        start = timeit.default_timer()

        # # Adjust these parameters for your CNN
        # n_filters = random.choice([16, 32, 64, 128, 256])
        # filter_size = random.choice([3, 5, 7, 9])
        # dropout = 0.01 * random.choice(range(5, 21))
        # activation_function = random.choice([torch.nn.Tanh(), torch.nn.ReLU(), torch.nn.ELU(), torch.nn.Mish()])
        # optimizer = random.choice([torch.optim.Adam, torch.optim.AdamW])
        # criterion = random.choice([torch.nn.MSELoss, torch.nn.L1Loss, torch.nn.SmoothL1Loss])
        # n_epochs = random.choice(range(200, 301))
        # batch_size = random.choice(range(256, 257))
        # learning_rate = 0.00001 * random.choice(range(5, 51))
        # patience = random.choice(range(1, 2))
        # min_delta = 0.0001 * random.choice(range(1, 2))
        # l1_regularization = 0.0000001 * random.choice(range(5, 21))
        # l2_regularization = l1_regularization
        # sequence_length = random.choice(range(5, 6)) 
         
        # Adjust these parameters for your CNN
        n_filters = random.choice(range(256, 512))
        filter_size = random.choice([3, 5])
        dropout = 0.01 * random.choice(range(5, 21))
        activation_function = random.choice([torch.nn.Tanh(), torch.nn.ReLU(), torch.nn.ELU(), torch.nn.Mish()])
        optimizer = random.choice([torch.optim.Adam, torch.optim.AdamW])
        criterion = random.choice([torch.nn.SmoothL1Loss])
        n_epochs = random.choice(range(200, 301))
        batch_size = random.choice(range(256, 257))
        learning_rate = 0.00001 * random.choice(range(10, 201))
        patience = random.choice(range(3, 4))
        min_delta = 0.0001 * random.choice(range(10, 21))
        l1_regularization = 0.0000001 * random.choice(range(5, 21))
        l2_regularization = l1_regularization
        sequence_length = random.choice(range(5, 6))

        # # Construct the full file paths
        # train_data_file_path = os.path.join(output_file_path, 'train_data.csv')
        # valid_data_file_path = os.path.join(output_file_path, 'valid_data.csv')

        # # Read the CSV files into Pandas DataFrames
        # train_data = pd.read_csv(train_data_file_path)
        # valid_data = pd.read_csv(valid_data_file_path)

        # X_train_tensor = torch.tensor(train_data.values, dtype=torch.float32)
        # X_valid_tensor = torch.tensor(valid_data.values, dtype=torch.float32)

        # X_train = create_sequences(X_train_tensor, sequence_length)
        # X_valid = create_sequences(X_valid_tensor, sequence_length)

        # y_train = X_train_tensor[sequence_length:, :]
        # y_valid = X_valid_tensor[sequence_length:, :]
        
        # Load tensors
        X_train = torch.load(os.path.join(output_file_path, 'X_train.pt'))
        y_train = torch.load(os.path.join(output_file_path, 'y_train.pt'))
        X_valid = torch.load(os.path.join(output_file_path, 'X_valid.pt'))
        y_valid = torch.load(os.path.join(output_file_path, 'y_valid.pt'))

        # Initialize the CNN model
        model = CNNModel(n_features=X_train.shape[2], n_filters=n_filters, filter_size=filter_size, dropout=dropout, activation_function=activation_function, 
                        l1_regularization=l1_regularization, l2_regularization=l2_regularization, sequence_length=sequence_length)
        
        # Train the model with the adjusted function for CNN
        train_losses, val_losses, stopped_early = train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=optimizer, criterion=criterion,
                                                                   batch_size=batch_size, patience=patience, min_delta=min_delta, learning_rate=learning_rate, trial=trial, save_directory=save_directory)

        # Check if training stopped early due to NaNs or not
        if stopped_early:
            print(f"Random search iteration {trial+1} stopped early due to NaNs in loss")
            # Using 'continue' here will skip the remaining statements of the current iteration and proceed to the next iteration
            continue

        if plot_loss:
            plot_results(train_losses, val_losses, trial, save_directory=save_directory)

        # # Save the model
        # if model_save:
        #     if save_directory:
        #         save_path = os.path.join(save_directory, f"model_trial_{trial}.pt")
        #     else:
        #         save_path = f"model_trial_{trial}.pt"
        #     torch.save(model, save_path)

        # For example, adjust parameters to log for CNN
        params = {
            "n_filters": n_filters, "filter_size": filter_size, "activation_function": activation_function,
                "optimizer": optimizer, "criterion": criterion, "dropout": dropout, "n_epochs": n_epochs,
                "batch_size": batch_size, "learning_rate": learning_rate, "patience": patience, "min_delta": min_delta,
                "l1_regularization": l1_regularization, "l2_regularization": l2_regularization,
        }

        results_params = {**params, "trial": trial+1, "train loss": train_losses[-1], "valid loss": val_losses[-1]}

        all_results_params.append(results_params)
        all_results_params_df = pd.DataFrame(all_results_params)

        if save_directory:
            all_results_params_df.to_csv(os.path.join(save_directory, f"all_results_params_{trial}.csv"))

        end = timeit.default_timer()
        # Calculate and print duration
        duration = end - start
        print(f"Execution Time of Trial {trial + 1} of {n_trials} is: {duration} seconds")

    return all_results_params_df


In [ ]:
n_trials = 3
# validation_frequency = 2
# batch_size_data = 100000

save_directory = "/home/predict_price/stock_price/save_model/CNN/ver4/ver4_no_target/ver4_seq_5_split_0.7_World_list_2001"
# output_file_path_params = "/home/predict_price/stock_price/save_model/ver5.8/ver5.8_decoder_no_split/ver5.8_decoder_no_split"
output_file_path = "/home/predict_price/stock_price/data_save/CNN/ver4/ver4_no_target/ver4_seq_5_split_0.7_World_list_2001"
all_results_params_df = model_save(n_trials=n_trials, model_save=True, save_directory=save_directory, plot_loss=True, output_file_path=output_file_path)
# Save results_df
all_results_params_df.to_csv(f'{save_directory}_all_results_params_df.csv', index=True)